In [1]:
import os
import pandas as pd

from collections import defaultdict
from torch.utils.data import DataLoader
from sklearn.neighbors import KNeighborsClassifier
from configs import configs
from dataset import ChestXRayCaptionDataset
import torch
import numpy as np
from model import Chexnet
from tqdm import tqdm
from utils import train_transform, evaluate_transform, quantize_probs
from tokenizer import create_tokenizer
from test import evaluation_matrix
from chexpert import chexpert
import time
import faiss

2091lines [00:00, 174290.34lines/s]


caller: c:\Users\darkenstardragon\Documents\Work\chest-xray-report-gen\text_generation\chexpert.py
Creating Chexpert reward module...
Using 1 GPUs!


In [2]:
LOAD_RANDOM_PROJECTION_DATA = False
BUILD_CACHED_MAPS = False
USE_CACHED_MAPS = True
SAVE_PROJECTED = True
time_file_path = 'results/inference_time.csv'

def filename(k, seed, project_dim):
    d = {
        'train_x': configs['mimic_dir'] + 'baseline_data/' +  f'train_image_embeddings_{project_dim}_{seed}.npy',
        'train_y': configs['mimic_dir'] + 'baseline_data/' +  f'train_captions.npy',
        'val_x': configs['mimic_dir'] +'baseline_data/' +  f'val_image_embeddings_{project_dim}_{seed}.npy',
        'val_y': configs['mimic_dir'] +'baseline_data/' +  f'val_captions.npy',
        'test_x': configs['mimic_dir'] +'baseline_data/' +  f'test_image_embeddings_{project_dim}_{seed}.npy',
        'test_y': configs['mimic_dir'] +'baseline_data/' +  f'test_captions.npy',
    }

    return d[k]


In [3]:
tokenizer = create_tokenizer()
checkpoint = torch.load('weights/pretrained_encoder/pretrained_enc_epoch_5_2022-03-08_15-43-47.540586.pth.tar')
print(f"loaded epoch {checkpoint['epoch']+1} model, val_loss: {checkpoint['val_loss']}")
encoder = checkpoint['encoder'].cuda()

train_probs_quantized = np.load(configs['mimic_dir'] + 'baseline_data/train_probs_quantized.npy')
val_probs_quantized = np.load(configs['mimic_dir'] + 'baseline_data/val_probs_quantized.npy')
test_probs_quantized = np.load(configs['mimic_dir'] + 'baseline_data/test_probs_quantized.npy')


train_loader = DataLoader(
    ChestXRayCaptionDataset('train', transform=train_transform),
    batch_size=16,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
)

val_loader = DataLoader(
    ChestXRayCaptionDataset('val', transform=evaluate_transform),
    batch_size=16,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
)

test_loader = DataLoader(
    ChestXRayCaptionDataset('test', transform=evaluate_transform),
    batch_size=16,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
)

len_train_loader = 16740
len_val_loader = 131
len_test_loader = 229

2091lines [00:00, 174273.02lines/s]


loaded epoch 5 model, val_loss: 0.28202417492866516


In [4]:
def generate_image_embeddings_random(encoder, data_loader, projection_matrix, project_every=2):
    # With random projection
    encoder.eval()
    image_embeddings = []
    captions = []
    batch = []
    with torch.no_grad():
        for i, (img, caption, _) in enumerate(tqdm(data_loader)):
            img = img.cuda()
            encoded_img, _ = encoder(img)
            batch.append(encoded_img.cpu())
            captions.append(caption.cpu())
            if ((i+1) % project_every) == 0 or (i+1) == len(data_loader):
                batch = torch.cat(batch).reshape(-1, 1024*8*8).numpy()
                batch = np.matmul(batch, projection_matrix)
                image_embeddings.append(batch)
                batch = []

    image_embeddings = np.vstack(image_embeddings)
    captions = torch.cat(captions).numpy()
    return image_embeddings, captions

def generate_image_embeddings_save_every(encoder, data_split, data_loader, save_every=1024):
    encoder.eval()
    image_embeddings = []
    captions = []
    file_index = 0

    with torch.no_grad():
        for i, (img, caption, _) in enumerate(tqdm(data_loader)):
            img = img.cuda()
            encoded_img, _ = encoder(img)
            image_embeddings.append(encoded_img.cpu())
            captions.append(caption.cpu())

            if ((i+1) % save_every) == 0 or (i+1) == len(data_loader):
                # stack
                image_embeddings = torch.cat(image_embeddings).reshape(-1, 1024*8*8).numpy()
                captions = torch.cat(captions).numpy()

                # save
                np.save(configs['mimic_dir'] + f'raw_embeddings/{data_split}/feature_maps_{file_index}.npy', image_embeddings)
                np.save(configs['mimic_dir'] + f'raw_embeddings/{data_split}/captions_{file_index}.npy', captions)

                # clear and update
                image_embeddings = []
                captions = []
                file_index += 1

def generate_probs_save_every(encoder, data_split, data_loader, save_every=1024):
    encoder.eval()
    probs = []
    file_index = 0

    with torch.no_grad():
        for i, (img, _, _) in enumerate(tqdm(data_loader)):
            img = img.cuda()
            _, prob = encoder(img)
            probs.append(prob.cpu())

            if ((i+1) % save_every) == 0 or (i+1) == len(data_loader):
                # stack
                probs = torch.cat(probs).numpy()

                # save
                np.save(configs['mimic_dir'] + f'raw_embeddings/{data_split}/probs_{file_index}.npy', probs)

                # clear and update
                probs = []
                file_index += 1

def random_project(len_data_loader, data_split, projection_matrix, save_every=1024, project_every=256):
    n_split = len_data_loader // save_every + 1
    print(f"{n_split=}")
    projected_image_embeddings = []
    captions = []
    
    for file_index in tqdm(range(n_split)):
        feat_maps = np.load(configs['mimic_dir'] + f'raw_embeddings/{data_split}/feature_maps_{file_index}.npy')
        caps = np.load(configs['mimic_dir'] + f'raw_embeddings/{data_split}/captions_{file_index}.npy')
        captions.append(caps)

        # project
        feat_maps = np.array_split(feat_maps, project_every)
        for batch in feat_maps:
            proj = np.matmul(batch, projection_matrix)
            projected_image_embeddings.append(proj)

    projected_image_embeddings = np.vstack(projected_image_embeddings)
    captions = np.vstack(captions)
    
    return projected_image_embeddings, captions

In [5]:
# generate_probs_save_every(encoder, 'train', train_loader, save_every=1024)
# generate_probs_save_every(encoder, 'val', val_loader, save_every=1024)
# generate_probs_save_every(encoder, 'test', test_loader, save_every=1024)

In [6]:
def get_vectors(SEED, RANDOM_PROJECT_DIM, load=False):
    """
    End to end train, val, test projected vectors function
    Input:
        SEED (int): Seed of the random projection matrix
        RANDOM_PROJECT_DIM: Dimension of the random projection matrix
    Output:
        train_x, train_y, val_x, val_y, test_x, test_y
    """

    """
    Load Cached Vectors
    """
    if load:
        
        file_suffix = f"_{RANDOM_PROJECT_DIM}_{SEED}.npy"
        file_prefix = configs['mimic_dir'] + 'baseline_data/'
        file_exists = os.path.exists(file_prefix + 'train_image_embeddings' + file_suffix)

        if file_exists:
            train_image_embeddings = np.load(file_prefix + 'train_image_embeddings' + file_suffix)
            train_captions = np.load(file_prefix + 'train_captions.npy')
            val_image_embeddings = np.load(file_prefix + 'val_image_embeddings' + file_suffix)
            val_captions = np.load(file_prefix + 'val_captions.npy')
            test_image_embeddings = np.load(file_prefix + 'test_image_embeddings' + file_suffix)
            test_captions = np.load(file_prefix + 'test_captions.npy')
            return train_image_embeddings, train_captions, val_image_embeddings, val_captions, test_image_embeddings, test_captions
        else:
            print(f"Attempting to load file with seed={SEED} and project_dim={RANDOM_PROJECT_DIM} but doesn't exist")
    
    print(f"Random projecting with {SEED=}, {RANDOM_PROJECT_DIM=}")
    # Create a whole new projection
    rng = np.random.RandomState(SEED)
    # Gaussian random projection
    projection_matrix = rng.normal(0.0, 1/RANDOM_PROJECT_DIM, (65536, RANDOM_PROJECT_DIM))

    """
    Train vectors
    """

    print("Projecting train vectors...")

    if USE_CACHED_MAPS:
        # New method: Predict first, cache them, then project
        if BUILD_CACHED_MAPS:
            generate_image_embeddings_save_every(encoder, 'train', train_loader, save_every=1024)

        train_image_embeddings, train_captions = random_project(len_train_loader, 'train', projection_matrix, save_every=1024, project_every=64)
        print(train_image_embeddings.shape)
        print(train_captions.shape)
        if SAVE_PROJECTED:
            np.save(filename('train_x', seed=SEED, project_dim=RANDOM_PROJECT_DIM), train_image_embeddings)
            np.save(filename('train_y', seed=SEED, project_dim=RANDOM_PROJECT_DIM), train_captions)
    else:
        # Old method: Project as we predict
        if LOAD_RANDOM_PROJECTION_DATA:
            # Use cached projection
            # train_image_embeddings = np.load(filename['train_x'])
            # train_captions = np.load(filename['train_y'])
            print(train_image_embeddings.shape)
            print(train_captions.shape)
        else:
            project_every = 256
            train_image_embeddings, train_captions = generate_image_embeddings_random(encoder, train_loader, projection_matrix, project_every=project_every)
            print(train_image_embeddings.shape)
            print(train_captions.shape)
            # np.save(filename['train_x'], train_image_embeddings)
            # np.save(filename['train_y'], train_captions)
    
    """
    Val & Test vectors
    """

    print("Projecting val & test vectors...")

    if USE_CACHED_MAPS:
        # New method: Predict first, cache them, then project
        if BUILD_CACHED_MAPS:
            generate_image_embeddings_save_every(encoder, 'val', val_loader, save_every=1024)
            generate_image_embeddings_save_every(encoder, 'test', test_loader, save_every=1024)

        val_image_embeddings, val_captions = random_project(len_val_loader, 'val', projection_matrix, save_every=1024, project_every=64)
        print(val_image_embeddings.shape)
        print(val_captions.shape)
        if SAVE_PROJECTED:
            np.save(filename('val_x', seed=SEED, project_dim=RANDOM_PROJECT_DIM), val_image_embeddings)
            np.save(filename('val_y', seed=SEED, project_dim=RANDOM_PROJECT_DIM), val_captions)
        test_image_embeddings, test_captions = random_project(len_test_loader, 'test', projection_matrix, save_every=1024, project_every=64)
        print(test_image_embeddings.shape)
        print(test_captions.shape)
        if SAVE_PROJECTED:
            np.save(filename('test_x', seed=SEED, project_dim=RANDOM_PROJECT_DIM), test_image_embeddings)
            np.save(filename('test_y', seed=SEED, project_dim=RANDOM_PROJECT_DIM), test_captions)

    else:
        if LOAD_RANDOM_PROJECTION_DATA:
            # val_image_embeddings = np.load(filename['val_x'])
            # val_captions = np.load(filename['val_y'])
            # test_image_embeddings = np.load(filename['test_x'])
            # test_captions = np.load(filename['test_y'])
            print(val_image_embeddings.shape)
            print(val_captions.shape)
            print(test_image_embeddings.shape)
            print(test_captions.shape)
        else:
            project_every = 256
            val_image_embeddings, val_captions = generate_image_embeddings_random(encoder, val_loader, projection_matrix, project_every=project_every)
            print(val_image_embeddings.shape)
            print(val_captions.shape)
            test_image_embeddings, test_captions = generate_image_embeddings_random(encoder, test_loader, projection_matrix, project_every=project_every)
            print(test_image_embeddings.shape)
            print(test_captions.shape)
            # np.save(filename['val_x'], val_image_embeddings)
            # np.save(filename['val_y'], val_captions)
            # np.save(filename['test_x'], test_image_embeddings)
            # np.save(filename['test_y'], test_captions)
    
    return train_image_embeddings, train_captions, val_image_embeddings, val_captions, test_image_embeddings, test_captions

In [7]:
class SimilaritySearch:
    def fit(self, xb, yb):
        pass

    def predict(self, xq):
        pass

class OneNearestNeighbor(SimilaritySearch):
    def __init__(self):
        self.yb = None
        self.knn = KNeighborsClassifier(n_neighbors=1)

    def fit(self, xb, yb):
        indices = [*range(xb.shape[0])]
        self.knn.fit(xb.astype(np.float32), indices)
        self.yb = yb.astype(np.float32)

    def predict(self, xq):
        dists, indices = self.knn.kneighbors(xq.astype(np.float32))
        yq = np.array([self.yb[i] for i in indices])
        yq = yq.reshape(xq.shape[0], self.yb.shape[1])
        return yq

class FaissFlatIndexL2CPU(SimilaritySearch):
    def __init__(self):
        self.yb = None
        self.index = None

    def fit(self, xb, yb):
        dim = xb.shape[1]
        self.index = faiss.IndexFlatL2(dim)
        self.index.add(xb.astype(np.float32))
        self.yb = yb.astype(np.float32)

    def predict(self, xq):
        dists, indices = self.index.search(xq.astype(np.float32), 1)
        yq = np.array([self.yb[i] for i in indices])
        yq = yq.reshape(xq.shape[0], self.yb.shape[1])
        return yq

class FaissFlatIndexL2GPU(SimilaritySearch):
    def __init__(self):
        self.res = faiss.StandardGpuResources()
        self.yb = None
        self.gpu_index = None

    def fit(self, xb, yb):
        dim = xb.shape[1]
        self.gpu_index = faiss.index_cpu_to_gpu(self.res, 0, faiss.IndexFlatL2(dim))
        self.gpu_index.add(xb.astype(np.float32))
        self.yb = yb.astype(np.float32)

    def predict(self, xq):
        dists, indices = self.gpu_index.search(xq.astype(np.float32), 1)
        yq = np.array([self.yb[i] for i in indices])
        yq = yq.reshape(xq.shape[0], self.yb.shape[1])
        return yq

class FaissHNSW32(SimilaritySearch):
    def __init__(self):
        self.yb = None
        self.index = None

    def fit(self, xb, yb):
        dim = xb.shape[1]
        self.index = faiss.IndexHNSWFlat(dim, 32)
        self.index.add(xb.astype(np.float32))
        self.yb = yb.astype(np.float32)

    def predict(self, xq):
        dists, indices = self.index.search(xq.astype(np.float32), 1)
        yq = np.array([self.yb[i] for i in indices])
        yq = yq.reshape(xq.shape[0], self.yb.shape[1])
        return yq

class FaissLSH32(SimilaritySearch):
    def __init__(self):
        self.yb = None
        self.index = None

    def fit(self, xb, yb):
        dim = xb.shape[1]
        self.index = faiss.IndexLSH(dim, 32)
        self.index.add(xb.astype(np.float32))
        self.yb = yb.astype(np.float32)

    def predict(self, xq):
        dists, indices = self.index.search(xq.astype(np.float32), 1)
        yq = np.array([self.yb[i] for i in indices])
        yq = yq.reshape(xq.shape[0], self.yb.shape[1])
        return yq

In [8]:
class SimilaritySearchCoarse2Fine(SimilaritySearch):
    def assign_labels(self, train_labels):
        self.train_labels = train_labels

    def assign_encoder(self, encoder):
        self.encoder = encoder

class OneNearestNeighborCoarse2Fine(SimilaritySearchCoarse2Fine):
    def __init__(self):
        # Dict label -> feature maps
        self.map = defaultdict(lambda: [])
        self.reports = defaultdict(lambda: [])
        self.knns = dict()
        self.assign_labels(train_probs_quantized)
    
    def fit(self, xb, yb):
        binary_strings = [''.join(label.astype(str)) for label in self.train_labels]
        # for label, feat_maps in tqdm(zip(binary_strings, xb), total=len(binary_strings)):
        for i in range(len(binary_strings)):
            label = binary_strings[i]
            feat_maps = xb[i]
            report = yb[i]
            self.map[label].append(feat_maps)
            self.reports[label].append(report)

        for label, feat_maps_list in self.map.items():
            indices = [*range(len(feat_maps_list))]
            self.knns[label] = KNeighborsClassifier(n_neighbors=1)
            self.knns[label].fit(np.array(feat_maps_list).astype(np.float32), indices)

    def predict(self, xq, x_image):
        _, probs = encoder(x_image)
        labels = quantize_probs(probs.detach().cpu().numpy())
        labels = [''.join(label.astype(str)) for label in labels]
        results = []
        no_label_count = 0
        for label, feat_map in zip(labels, xq):
            # Might not found exact label in training
            # Might need to search in near edit distance
            if label in self.knns.keys():
                dists, index = self.knns[label].kneighbors(feat_map.astype(np.float32).reshape(1, -1))
                yq = np.array(self.reports[label][index[0][0]]).reshape(-1)
            else:
                # Handle no label
                similar_labels_list = self.get_similar_binaries(label)
                similar_feature_maps = []
                similar_reports = []
                for sim_label in similar_labels_list:
                    if sim_label in self.map.keys():
                        # Need to make sure the sim label exists
                        similar_feature_maps.extend(self.map[sim_label])
                        similar_reports.extend(self.reports[sim_label])
                
                # Create temp knn
                temp_knn = KNeighborsClassifier(n_neighbors=1)
                indices = [*range(len(similar_feature_maps))]
                temp_knn.fit(similar_feature_maps, indices)
                dists, index = temp_knn.kneighbors(feat_map.astype(np.float32).reshape(1, -1))
                yq = np.array(similar_reports[index[0][0]]).reshape(-1)
                no_label_count += 1 # For monitoring purpose

            results.append(yq)

        # if no_label_count > 0:
        #     print(f"No label count: {no_label_count}")
        return np.array(results)

    def get_similar_binaries(self, binary_str: str):
        """
        Returns a list of similar binary string with edit distance = 1
        """
        res = []
        for i in range(len(binary_str)):
            res.append(binary_str[:i] + str(int(not(bool(int(binary_str[i]))))) + binary_str[i+1:])
        return res

In [9]:
def predict(model, embeddings, decode=False, batch_size=64, image_loader=None):
    captions = []
    total_time = 0.0

    assert isinstance(model, SimilaritySearchCoarse2Fine) == (image_loader != None), (
        f"isinstance={isinstance(model, SimilaritySearchCoarse2Fine)} but image_loader != None is {image_loader != None}"
    )

    if isinstance(model, SimilaritySearchCoarse2Fine) and image_loader != None:
        print(f"forced batch size: {image_loader.batch_size}")
        data_loader = DataLoader(
            embeddings,
            batch_size=image_loader.batch_size,
            num_workers=0,
            pin_memory=True,
        )

        assert len(data_loader) == len(image_loader), (
            f"{len(data_loader)=}, {len(image_loader)=}"
        )

        for j, (batch, (image, _, _)) in enumerate(tqdm(zip(data_loader, image_loader), total=len(data_loader))):
            start = time.time()
            yq = model.predict(batch.numpy(), image.cuda())
            total_time += time.time() - start
            captions.extend(yq)
    else:
        data_loader = DataLoader(
            embeddings,
            batch_size=batch_size,
            num_workers=0,
            pin_memory=True,
        )
        for j, batch in enumerate(tqdm(data_loader)):
            start = time.time()
            yq = model.predict(batch.numpy())
            total_time += time.time() - start
            captions.extend(yq)

    captions = np.array(captions).reshape(embeddings.shape[0], -1)
    if decode:
        captions = tokenizer.decode(captions)
    return captions, total_time

def evaluate(true_captions, pred_captions, batch_size):
    true_df = []
    pred_df = []
    print(true_captions.shape[0], pred_captions.shape[0])
    true_loader = DataLoader(
        true_captions, 
        batch_size=batch_size,
        num_workers=0,
        pin_memory=True
    )
    pred_loader = DataLoader(
        pred_captions, 
        batch_size=batch_size,
        num_workers=0,
        pin_memory=True
    )

    for t in tqdm(true_loader):
        labels = chexpert(t, tokenizer)
        true_df.append(labels)

    for p in tqdm(pred_loader):
        labels = chexpert(p, tokenizer)
        pred_df.append(labels)
    
    true_df = pd.concat(true_df).reset_index(drop=True)
    pred_df = pd.concat(pred_df).reset_index(drop=True)
    return evaluation_matrix(true_df, pred_df)

def write_time_file(model, project_dim, seed, val_time, test_time):
    with open(time_file_path, 'a') as f:
        model_type = type(model).__name__
        f.write(f"{model_type},{pd.Timestamp.now()},{project_dim},{seed},{val_time},{test_time}\n")

def evaluate_all(model, val_image_embeddings, val_captions, test_image_embeddings, test_captions, seed, project_dim):
    if isinstance(model, SimilaritySearchCoarse2Fine):
        predicted_reports, val_time = predict(model, val_image_embeddings, batch_size=1024, image_loader=val_loader)
    else:
        predicted_reports, val_time = predict(model, val_image_embeddings, batch_size=1024)
    print(f"Time taken to predict val: {val_time:.3f} seconds")
    val_eval_matrix = evaluate(val_captions, predicted_reports, batch_size=8)
    val_eval_matrix.to_csv(
        f'results/{type(model).__name__}_val_results_{project_dim}_{seed}.csv', index=False
    )
    print(val_eval_matrix)

    if isinstance(model, SimilaritySearchCoarse2Fine):
        predicted_reports, test_time = predict(model, test_image_embeddings, batch_size=1024, image_loader=test_loader)
    else:
        predicted_reports, test_time = predict(model, test_image_embeddings, batch_size=1024)
    print(f"Time taken to predict test: {test_time:.3f} seconds")
    test_eval_matrix = evaluate(test_captions, predicted_reports, batch_size=8)
    test_eval_matrix.to_csv(
        f'results/{type(model).__name__}_test_results_{project_dim}_{seed}.csv', index=False
    )
    print(test_eval_matrix)

    write_time_file(model, project_dim, seed, val_time, test_time)

In [10]:
def e2e_benchmark(model_class, seed, project_dim, load=False):
    """
    Perform vector acquisition, 1-NN, and evaluate on val and test set, and save results into files
    """

    # Get vector
    train_image_embeddings, train_captions, val_image_embeddings, val_captions, test_image_embeddings, test_captions = get_vectors(seed, project_dim, load=load)

    # Model selection
    model = model_class()
    if isinstance(model, SimilaritySearchCoarse2Fine):
        model.assign_encoder(encoder)

    # Train
    model.fit(train_image_embeddings, train_captions)

    # Evaluate
    evaluate_all(model, val_image_embeddings, val_captions, test_image_embeddings, test_captions, seed, project_dim)

In [ ]:
models = [OneNearestNeighborCoarse2Fine]
seeds = [0, 1000, 2000, 3000, 4000]
dims = [128, 256, 512, 1024, 2048, 4096, 8192]

for model_class in models:
    for dim in dims:
        for seed in seeds:
            path_to_check = f'results/{model_class.__name__}_test_results_{dim}_{seed}.csv'
            exist = os.path.exists(path_to_check)
            if not exist:
                print(model_class.__name__, dim, seed)
                e2e_benchmark(model_class, seed=seed, project_dim=dim, load=True)

In [19]:
def chexperify(captions, batch_size):
    """
    Returns a dataframe with labels and captions alongside
    Example usage:

        true_df = chexperify(test_captions, batch_size=12)
        pred_df = chexperify(test_predicted_reports, batch_size=12)

    """
    df = []
    data_loader = DataLoader(
        captions, 
        batch_size=batch_size,
        num_workers=0,
        pin_memory=True
    )

    for batch in tqdm(data_loader):
        labels = chexpert(batch, tokenizer)
        df.append(labels)


    df = pd.concat(df).reset_index(drop=True)
    df['captions'] = tokenizer.decode(captions)
    return df

### Test from remote run on server

Requirements: `predicted_val_captions.npy` and `predicted_test_captions.npy`

In [8]:
pred_captions = np.load('remote_server/predicted_val_captions.npy')
until = pred_captions.shape[0]
true_captions = np.load('mimic_cxr/raw_embeddings/val/captions_0.npy')[:until]
evaluate(true_captions, pred_captions, batch_size=12)

100%|██████████| 174/174 [01:05<00:00,  2.64it/s]


Metrics,Recall,Precision,F1
Enlarged Cardiomediastinum,0.729483,0.658436,0.692141
Cardiomegaly,0.657244,0.613861,0.634812
Lung Opacity,0.744152,0.712885,0.728183
Lung Lesion,0.266355,0.360759,0.306452
Edema,0.603448,0.550562,0.575793
Consolidation,0.620123,0.683258,0.650161
Pneumonia,0.438119,0.517544,0.474531
Atelectasis,0.624299,0.660079,0.641691
Pneumothorax,0.148148,0.171429,0.158940
Pleural Effusion,0.640426,0.650108,0.645230


In [12]:
pred_captions = np.load('remote_server/predicted_test_captions.npy')
until = pred_captions.shape[0]
true_captions = np.load('mimic_cxr/raw_embeddings/test/captions_0.npy')[:until]
evaluate(true_captions, pred_captions, batch_size=12)

100%|██████████| 42/42 [00:09<00:00,  4.50it/s]


Metrics,Recall,Precision,F1
Enlarged Cardiomediastinum,0.746429,0.741135,0.743772
Cardiomegaly,0.691304,0.679487,0.685345
Lung Opacity,0.784722,0.733766,0.758389
Lung Lesion,0.200000,0.367647,0.259067
Edema,0.565934,0.559783,0.562842
Consolidation,0.480952,0.554945,0.515306
Pneumonia,0.413408,0.544118,0.469841
Atelectasis,0.610465,0.486111,0.541237
Pneumothorax,0.071429,0.142857,0.095238
Pleural Effusion,0.527607,0.530864,0.529231
